In [1]:
pwd = %pwd
pwd[12:]

'PJATK/semestr4/ADD/Covid19_Prediction/notebooks'

In [2]:
import os
os.chdir("..")

In [3]:
pwd = %pwd
pwd[12:]

'PJATK/semestr4/ADD/Covid19_Prediction'

In [4]:
import pandas as pd

In [5]:
import json
with open("data/covid_data.json") as f:
    d = json.load(f)

In [6]:
df = pd.DataFrame(d["records"])

In [7]:
df.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2020,continentExp
0,23/10/2022,23,10,2022,3557.0,0.0,Austria,AT,AUT,8901064,Europe
1,22/10/2022,22,10,2022,5494.0,4.0,Austria,AT,AUT,8901064,Europe
2,21/10/2022,21,10,2022,7776.0,4.0,Austria,AT,AUT,8901064,Europe
3,20/10/2022,20,10,2022,8221.0,6.0,Austria,AT,AUT,8901064,Europe
4,19/10/2022,19,10,2022,10007.0,8.0,Austria,AT,AUT,8901064,Europe


In [8]:
"""todo create features -> 
is_święto, 
season(pora roku), 
death rate, 
R0 (https://en.wikipedia.org/wiki/Basic_reproduction_number)
"""

'todo create features -> \nis_święto, \nseason(pora roku), \ndeath rate, \nR0 (https://en.wikipedia.org/wiki/Basic_reproduction_number)\n'

In [9]:
df = df[df["countryterritoryCode"] == "POL"].sort_values("dateRep")

In [10]:
cleaned_data = df.drop(["countryterritoryCode", "continentExp", "geoId", "countriesAndTerritories", "dateRep", "day", "month", "year"], axis=1)

In [11]:
cleaned_data.head()

,cases,deaths,popData2020
22735,10896.0,402.0,37958138
22370,12033.0,505.0,37958138
22704,2504.0,42.0,37958138
22339,39116.0,240.0,37958138
22676,4786.0,24.0,37958138


In [14]:
K = 5 # sliding window range

In [15]:
cleaned_data["window_sum"] = cleaned_data["cases"].rolling(K).sum()
cleaned_data["window_min"] = cleaned_data["cases"].rolling(K).min()
cleaned_data["window_max"] = cleaned_data["cases"].rolling(K).max()
cleaned_data["window_std"] = cleaned_data["cases"].rolling(K).std()
cleaned_data["window_var"] = cleaned_data["cases"].rolling(K).var()
cleaned_data["window_mean"] = cleaned_data["cases"].rolling(K).mean()
cleaned_data["window_median"] = cleaned_data["cases"].rolling(K).median()
cleaned_data["window_delta"] = cleaned_data["window_max"] - cleaned_data["window_min"]

In [16]:
cleaned_data['cases_next_day'] = df['cases'].shift(1)

In [17]:
cleaned_data

,cases,deaths,popData2020,window_sum,window_min,window_max,window_std,window_var,window_mean,window_median,window_delta,cases_next_day
22735,10896.0,402.0,37958138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22370,12033.0,505.0,37958138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10896.0
22704,2504.0,42.0,37958138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12033.0
22339,39116.0,240.0,37958138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2504.0
22676,4786.0,24.0,37958138,69335.0,2504.0,39116.0,14673.826427,215321182.0,13867.0,10896.0,36612.0,39116.0
...,...,...,...,...,...,...,...,...,...,...,...,...
22128,3102.0,34.0,37958138,4615.0,153.0,3102.0,1229.606238,1511931.5,923.0,502.0,2949.0,285.0
22797,21897.0,280.0,37958138,26359.0,285.0,21897.0,9364.870458,87700798.7,5271.8,573.0,21612.0,3102.0
22432,7147.0,9.0,37958138,32933.0,285.0,21897.0,8994.405945,80899338.3,6586.6,3102.0,21612.0,21897.0
22736,13464.0,535.0,37958138,45895.0,285.0,21897.0,8664.163808,75067734.5,9179.0,7147.0,21612.0,7147.0


In [18]:
x = cleaned_data.drop(["cases_next_day"], axis=1).values
y = cleaned_data["cases_next_day"].values